In [1]:
import pandas as pd
jsonObj = pd.read_json(path_or_buf="descriptions/VerilogDescription_Human.jsonl", lines=True)

In [2]:
print(jsonObj.head())

        task_id                                 detail_description
0      mux2to1v  Create a 2-1 multiplexer. When sel=0, choose a...
1     m2014_q6b  Consider the state machine shown below:\n\n// ...
2        ringer  Suppose you are designing a circuit to control...
3  alwaysblock1  Build an AND gate using both an assign stateme...
4          zero         Build a circuit that always outputs a LOW.


In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
peft_model_id = "/home/daniel.kiesewalter/project_aicd_self_train/results/final_checkpoint"

model = AutoModelForCausalLM.from_pretrained(model_id)
model.load_adapter(peft_model_id)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",  # if you have GPU
)

terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [35]:
designs = []
for index, row in jsonObj.iterrows():
    chat = [
        {"role": "system", "content": "You are a digital designer that creates modules in system verilog on request. Only return the desired system verilog code and do not include any descrition of the code or further text."},
        {"role":"user", "content": row["detail_description"]}]
    message = pipe.tokenizer.apply_chat_template(chat, add_generation_prompt=True, tokenize=False)
    generated_text=pipe(
        message,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        # top_k=10,
        top_p=0.9,
        return_full_text=False,
        max_new_tokens=3000
        )[0]["generated_text"]
    design = {
        "task_id": row["task_id"],
        "completion": generated_text
        }
    designs.append(design)
    # break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [1]:
df = pd.Dataframe(designs)
df.to_json("samples.jsonl", lines=True)

NameError: name 'pd' is not defined

In [ ]:
! mkdir -p build/
! cd build/
! ../configure  --with-task=spec-to-rtl --with-model=meta-llama/Meta-Llama-3-8B-Instruct #--with-examples=$shots --with-samples=$samples --with-temperature=$temperature --with-top-p=$top_p
! make
